# Baskerville HPC CO2 emissions

In [ ]:
import pandas as pd
import yaml

from src.energy import get_energy_cpu, get_energy_gpu, get_energy_mem
from src.ci_calculator import CarbonIntensityCalculator

## Load data

In [ ]:
# load Baskerville usage data
data = pd.read_csv("./data/Baskerville_total_commas.csv")
data["End Time"] = pd.to_datetime(data["End Time"])

# load Baskerville HPC config
cluster_info = yaml.safe_load(open("./data/hpc_config.yaml"))

In [ ]:
# for aggregating by month/year
data["YYYY-MM"] = data["End Time"].apply(lambda x: x.strftime("%Y-%m"))
data["YYYY"] = data["End Time"].apply(lambda x: x.strftime("%Y"))

## Calculate energies

In [ ]:
# calculate energies
data["E_cpu"] = data.apply(axis=1, func=get_energy_cpu, cluster_info=cluster_info) 
data["E_gpu"] = data.apply(axis=1, func=get_energy_gpu, cluster_info=cluster_info)
data["E_mem"] = data.apply(axis=1, func=get_energy_mem, cluster_info=cluster_info)
data["E_total_compute"] = data["E_cpu"] + data["E_gpu"] + data["E_mem"]
data["E_total"] = data["E_total_compute"] * cluster_info["PUE"] # PUE is ratio, still in kWh

In [ ]:
data.groupby("YYYY")["E_total_compute"].sum().plot(
    kind="bar", 
    title="Baskerville Energy Usage", 
    ylabel="kWh", 
    xlabel="Year", 
    figsize=(10,5)
)

## Calculate emissions using gas powered CHP carbon intensity

Noussan et al, 2024, Hourly electricity CO2 intensity profiles based on the real operation of large-scale natural gas combined cycle cogeneration plants, https://doi.org/10.1016/j.energy.2024.133424.

In [ ]:
data["CO2_total_g_CHP"] = data["E_total"] * 300

In [ ]:
data.to_csv("./data/Baskerville_ci_gas_chp.csv")

In [ ]:
print(f"All time CO2 emissions using gas CHP CI: {data['CO2_total_g_CHP'].sum() / 1e6 :.2f} tonnes") # in tonnes CO2

In [ ]:
print(f"Past 12 months CO2 emissions using gas CHP CI: {data.groupby('YYYY-MM')['CO2_total_g_CHP'].sum()['2024-09':].sum() / 1e6 :.2f} tonnes") # in tonnes CO2

In [ ]:
data.groupby("YYYY")["CO2_total_g_CHP"].sum().apply(lambda x: x / 1e6).to_dict() # in tonnes CO2

## Calculate emissions using yearly average carbon intensity

In [ ]:
data["CO2_total_g_yearly"] = data["E_total"] * 124 # UK average 2024

In [ ]:
data.to_csv("./data/Baskerville_ci_average_2024.csv")

In [ ]:
print(f"All time CO2 emissions using UK average CI: {data['CO2_total_g_yearly'].sum() / 1e6 :.2f} tonnes") # in tonnes CO2

In [ ]:
print(f"Past 12 months CO2 emissions using UK average CI: {data.groupby('YYYY-MM')['CO2_total_g_yearly'].sum()['2024-09':].sum() / 1e6 :.2f} tonnes") # in tonnes CO2

In [ ]:
data.groupby("YYYY")["CO2_total_g_yearly"].sum().apply(lambda x: x / 1e6).to_dict() # in tonnes CO2

## Calculate emissions using daily average carbon intensity

In [ ]:
# calculate daily average CI for each job
ci_calculator = CarbonIntensityCalculator()
data["CI"] = ci_calculator.get_carbon_intensity(data["End Time"])

In [ ]:
data["CO2_total_g_daily"] = data["E_total"] * data["CI"] # daily average

In [ ]:
data.to_csv("./data/Baskerville_ci_daily_ci.csv")

In [ ]:
print(f"All time CO2 emissions using daily average CI: {data['CO2_total_g_daily'].sum() / 1e6 :.2f} tonnes") # in tonnes CO2

In [ ]:
print(f"Past 12 months CO2 emissions using daily average CI: {data.groupby('YYYY-MM')['CO2_total_g_daily'].sum()['2024-09':].sum() / 1e6 :.2f} tonnes") # in tonnes CO2

In [ ]:
yearly_averages = data.groupby("YYYY")["CO2_total_g_daily"].sum() / 1e6 # in tonnes CO2
yearly_averages = yearly_averages.to_dict()

yearly_averages

In [ ]:
ax = data.groupby("YYYY")[["CO2_total_g_CHP", "CO2_total_g_yearly", "CO2_total_g_daily"]].sum().apply(lambda x: x/1e6).plot(
    kind="bar", 
    title="Baskerville Yearly Emissions", 
    ylabel="tonnes CO2e", 
    xlabel="Year", 
    figsize=(10,5),
)
ax.legend(["Gas CHP CI", "UK Average 2024 CI", "Daily Average CI"])